# Building a Multi-Layer Perceptron (MLP) with NumPy

In the previous [notebook](./neuron_using_numpy.ipynb), we implemented a single neuron from scratch using NumPy. We extended this to build a simple neural network with a single hidden layer that could process either a single input sample or a batch of samples.

In this notebook, we take the next logical step by building a **Multi-Layer Perceptron (MLP)**—a feedforward neural network that consists of multiple layers, each with a customizable number of neurons. The MLP we construct here will be fully dynamic and can handle the following input configurations:

---

## 💡 Capabilities of the MLP Implementation

The function you’ll build is designed to handle the following scenarios:

1. **Single scalar input** (e.g., `3.2`)
2. **Single input vector** (e.g., `[1.0, 2.0, 3.0]`)
3. **Batch of input vectors** (e.g., `[[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]]`)
4. **Any number of hidden layers**, followed by the output layer
5. **Variable number of neurons per layer**, with each layer having its own weight matrix and bias vector

---

## 🔧 Assumptions

- The only external dependency is `NumPy`, which is a standard library for numerical computation in Python.
- You can install it via:
  ```bash
  pip install numpy
  ```

---

## 📥 Expected User Inputs

The user needs to provide the following:

1. **Input (`x`)**: Can be:
    - A float (scalar input)
    - A list of floats (single input vector)
    - A list of lists of floats (batch of input vectors)

2. **`num_layers`**: An integer indicating the total number of layers (including both hidden and output layers).

3. **Weights and Biases (per layer)**:
    - Each layer’s **weights** and **biases** are to be provided as additional arguments.
    - These are passed dynamically using the `*args` mechanism.
    - The sequence should strictly alternate: `weights_layer1, bias_layer1, weights_layer2, bias_layer2, ..., weights_layerN, bias_layerN`.

---

## 🧠 Interpreting Weight and Bias Shapes

The user should follow these conventions when providing weights and biases:

### Weights:
- If the input is a **scalar**, the weights for the first layer should be a list (e.g., `[0.5]` for 1 neuron).
- If the input is a **vector of size `n`**, the weights for one neuron should be a list of length `n`, and for `m` neurons, a list of `m` such lists, i.e., a 2D list of shape `(m, n)`.
- If the input is a **batch of vectors**, the weights should still follow the shape `(neurons, features)`, matching the input feature dimension.

### Biases:
- Each bias should be a list of floats of length equal to the number of neurons in that layer.
- Scalar biases can be provided as a single float, in which case it will be broadcasted across all neurons in that layer.

---

## ✅ Example Usage

Let’s assume we want to build a 2-layer MLP (1 hidden layer with 3 neurons, and 1 output layer with 2 neurons):

```python
# Input: batch of 2 samples, each with 4 features
input = [[0.1, 0.2, 0.3, 0.4],
         [0.5, 0.6, 0.7, 0.8]]

num_layers = 2

# Layer 1: 3 neurons, each takes 4 input features
weights1 = [[0.1, 0.2, 0.3, 0.4],
            [0.5, 0.6, 0.7, 0.8],
            [0.9, 1.0, 1.1, 1.2]]
bias1 = [0.1, 0.2, 0.3]

# Layer 2: 2 neurons, each takes 3 input features (from previous layer)
weights2 = [[0.1, 0.2, 0.3],
            [0.4, 0.5, 0.6]]
bias2 = [0.1, 0.2]
```

These would be passed to the function as:

```python
output = forward_pass(input, num_layers, weights1, bias1, weights2, bias2)
```

---

## 🔁 Function Design: `mlp_using_numpy`

The core function will use `*args` to accept dynamic weights and biases for all layers. It will:

- Normalize the input into a NumPy array
- Iterate through each layer
- Apply a linear transformation (`z = x @ W.T + b`)
- Use an activation function like ReLU (or identity for output) after each layer

---

## ⚠️ Error Handling Considerations

The implementation should validate the following:

- Total number of weight-bias pairs must match `num_layers * 2`.
- Weight matrices should align with the dimensions of the input or previous layer’s output.
- Bias vectors must match the number of neurons in their respective layers.
- Input shapes are validated and reshaped where necessary for consistency.



---

By the end of this notebook, you will have a flexible and modular MLP architecture that is built entirely from scratch using NumPy, with full support for variable input dimensions and arbitrary depth.


In [1]:
# Importing necessary libraries

import numpy as np
from typing import List, Union

In [ ]:
def mlp_using_numpy(
    inputs: Union[float, List[float], List[List[float]]], num_layers: int, *args
) -> Union[float, List[float], List[List[float]]]:
    """
    Implements a simple Multi-Layer Perceptron (MLP) using NumPy, without activation functions.
    Supports scalar, vector, or batch input and arbitrary number of layers with custom weights/biases.

    The function expects arguments in the following sequence:
    - inputs: scalar / list / list of lists
    - num_layers: total number of layers (hidden + output)
    - *args: A flat sequence alternating weights and biases for each layer:
        (weights1, bias1, weights2, bias2, ..., weightsN, biasN)

    Each weight should be a 2D list (neurons_in_layer, input_dim),
    and each bias should be a 1D list (length = neurons_in_layer).

    Args:
        inputs (Union[float, List[float], List[List[float]]]):
            Input to the network. Can be:
                - float: scalar input
                - list of floats: single sample
                - list of list of floats: batch of samples
        num_layers (int): Number of layers in the MLP (including output layer)
        *args: Weights and biases for each layer, alternating as
               weights1, bias1, weights2, bias2, ..., weightsN, biasN.

    Returns:
        Union[float, List[float], List[List[float]]]: Output of the MLP
            - float if single scalar output
            - list of floats for single sample
            - list of list of floats for batch output

    Raises:
        ValueError: If shape mismatches or incorrect number of arguments are detected.

    Example:
        >>> mlp_using_numpy([1.0, 2.0], 2,
        ...                 [[0.5, 0.5], [0.5, 0.5]], [0.1, 0.1],
        ...                 [[0.3, 0.3], [0.3, 0.3]], [0.2, 0.2])
        [1.16, 1.16]
    """

    # Convert inputs to NumPy array and reshape if needed
    inputs = np.array(inputs, dtype=float)

    if inputs.ndim == 0:
        inputs = inputs.reshape(1, 1)  # scalar input
    elif inputs.ndim == 1:
        inputs = inputs.reshape(1, -1)  # single input vector
    elif inputs.ndim == 2:
        pass  # batch input
    else:
        raise ValueError("Input must be a scalar, 1D list, or 2D list.")

    # Validate number of layers
    if num_layers < 1:
        raise ValueError("Number of layers must be at least 1.")

    # Validate number of weight/bias arguments
    if len(args) != 2 * num_layers:
        raise ValueError(
            f"Expected {2 * num_layers} arguments for {num_layers} layers (weights and biases), but got {len(args)}."
        )

    # Forward pass through each layer
    output = inputs
    for layer in range(num_layers):
        weights = np.array(args[2 * layer], dtype=float)
        biases = np.array(args[2 * layer + 1], dtype=float)

        if weights.ndim != 2:
            raise ValueError(f"Layer {layer + 1} weights must be a 2D list or array.")
        if biases.ndim != 1:
            raise ValueError(f"Layer {layer + 1} biases must be a 1D list or array.")

        if weights.shape[1] != output.shape[1]:
            raise ValueError(
                f"Shape mismatch at layer {layer + 1}: weight expects input of shape (*, {weights.shape[1]}), got (*, {output.shape[1]})."
            )

        if biases.shape[0] != weights.shape[0]:
            raise ValueError(
                f"Shape mismatch: bias length ({biases.shape[0]}) does not match number of neurons ({weights.shape[0]}) at layer {layer + 1}."
            )

        # Forward step: output = input @ weights.T + bias
        output = output @ weights.T + biases

    # Output formatting
    if output.shape[0] == 1:
        if output.shape[1] == 1:
            return float(output[0, 0])  # scalar output
        return output.flatten().tolist()  # single sample output
    else:
        return output.tolist()  # batch output


In [ ]:
output = mlp_using_numpy(
    [1.0, 2.0],
    2,
    [[0.5, 0.5], [0.5, 0.5]],
    [0.1, 0.1],
    [[0.3, 0.3], [0.3, 0.3]],
    [0.2, 0.2],
)

print(output)

[1.16, 1.16]
